In [5]:
!pip install keras

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import matplotlib.pyplot as plt
import os

In [21]:
train_dir = 'data/train'
validation_dir = 'data/validation'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical')

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical')


Found 7143 images belonging to 3 classes.
Found 0 images belonging to 3 classes.


In [16]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_6 (Conv2D)                    │ (None, 148, 148, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_6 (MaxPooling2D)       │ (None, 74, 74, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 72, 72, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_7 (MaxPooling2D)       │ (None, 36, 36, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_8 (Conv2D)                    │ (None, 34, 34, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_8 (MaxPooling2D)       │ (None, 17, 17, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_2 (Flatten)                  │ (None, 36992)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 512)                 │      18,940,416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_2 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 3)                   │           1,539 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 19,035,203 (72.61 MB)

 Trainable params: 19,035,203 (72.61 MB)

 Non-trainable params: 0 (0.00 B)

In [22]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=30)

Epoch 1/30
223/223 ━━━━━━━━━━━━━━━━━━━━ 0s 476ms/step - accuracy: 0.9357 - loss: 0.1946

ValueError: Must provide at least one structure

In [20]:
import os
from PIL import Image

def check_images(directory):
    corrupted_images = []
    for root, _, files in os.walk(directory):
        for file in files:
            try:
                img_path = os.path.join(root, file)
                img = Image.open(img_path)
                img.verify()
            except (IOError, SyntaxError) as e:
                print(f"Corrupted image detected: {img_path}")
                corrupted_images.append(img_path)
    return corrupted_images

train_dir = 'data/train'
validation_dir = 'data/validation'

corrupted_train_images = check_images(train_dir)
corrupted_validation_images = check_images(validation_dir)

print(f"Found {len(corrupted_train_images)} corrupted images in training directory.")
print(f"Found {len(corrupted_validation_images)} corrupted images in validation directory.")

for img_path in corrupted_train_images + corrupted_validation_images:
    os.remove(img_path)
    print(f"Removed corrupted image: {img_path}")


Found 0 corrupted images in training directory.
Found 0 corrupted images in validation directory.


In [25]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

classifier = Sequential()

classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))

classifier.add(MaxPooling2D(pool_size=(2, 2)))

classifier.add(Conv2D(32, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

classifier.add(Flatten())

classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dropout(0.5))
classifier.add(Dense(units=3, activation='softmax'))

classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [30]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

validation_set = test_datagen.flow_from_directory(
    'data/validation',
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical')

# Define the number of images for each class
num_air_pollution_images = 6528
num_water_pollution_images = 322
num_soil_pollution_images = 301

# Calculate the steps per epoch for each class
steps_per_epoch_air_pollution = num_air_pollution_images // 32
steps_per_epoch_water_pollution = num_water_pollution_images // 32
steps_per_epoch_soil_pollution = num_soil_pollution_images // 32

# Calculate the total steps per epoch for the training set
steps_per_epoch_total = (
    steps_per_epoch_air_pollution +
    steps_per_epoch_water_pollution +
    steps_per_epoch_soil_pollution
)

# Load the training data using ImageDataGenerator
training_set = train_datagen.flow_from_directory(
    'data/train',
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical'
)

# Train the model using the calculated steps per epoch
classifier.fit(
    training_set,
    steps_per_epoch=steps_per_epoch_total,
    epochs=25
)


Found 0 images belonging to 3 classes.
Found 7143 images belonging to 3 classes.
Epoch 1/25
223/223 ━━━━━━━━━━━━━━━━━━━━ 28s 123ms/step - accuracy: 0.9295 - loss: 0.1825
Epoch 2/25
223/223 ━━━━━━━━━━━━━━━━━━━━ 0s 22us/step - accuracy: 0.9375 - loss: 0.2522
Epoch 3/25
223/223 ━━━━━━━━━━━━━━━━━━━━ 29s 129ms/step - accuracy: 0.9436 - loss: 0.1601
Epoch 4/25
223/223 ━━━━━━━━━━━━━━━━━━━━ 0s 18us/step - accuracy: 0.9062 - loss: 0.1712
Epoch 5/25
223/223 ━━━━━━━━━━━━━━━━━━━━ 29s 127ms/step - accuracy: 0.9482 - loss: 0.1421
Epoch 6/25
223/223 ━━━━━━━━━━━━━━━━━━━━ 0s 25us/step - accuracy: 0.9375 - loss: 0.1853
Epoch 7/25
223/223 ━━━━━━━━━━━━━━━━━━━━ 29s 128ms/step - accuracy: 0.9560 - loss: 0.1263
Epoch 8/25
223/223 ━━━━━━━━━━━━━━━━━━━━ 0s 23us/step - accuracy: 0.9062 - loss: 0.2317
Epoch 9/25
223/223 ━━━━━━━━━━━━━━━━━━━━ 29s 127ms/step - accuracy: 0.9544 - loss: 0.1256
Epoch 10/25
223/223 ━━━━━━━━━━━━━━━━━━━━ 0s 23us/step - accuracy: 0.9688 - loss: 0.0726
Epoch 11/25
223/223 ━━━━━━━━━━━━━━━━━━

In [32]:
# Load a small subset of training data for predictions
sample_images, _ = next(training_set)
predictions = classifier.predict(sample_images)
predicted_classes = np.argmax(predictions, axis=1)

# Print the predicted classes
print("Predicted classes:", predicted_classes)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
Predicted classes: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 2 0]


In [35]:
# Load a small subset of training data for predictions
sample_images, _ = next(training_set)

# Generate predictions for the sample images
predictions = classifier.predict(sample_images)
predicted_classes = np.argmax(predictions, axis=1)

# Analyze the distribution of predicted classes
class_counts = np.bincount(predicted_classes)
print("Predicted class distribution:", class_counts)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
Predicted class distribution: [29  2  1]


In [38]:
# Load a small subset of training data for predictions
sample_images, _ = next(training_set)

# Generate predictions for the sample images
predictions = classifier.predict(sample_images)
predicted_classes = np.argmax(predictions, axis=1)

# Print the indices and predicted classes of all images in the sample batch
for idx, pred_class in enumerate(predicted_classes):
    print(f"Image index {idx}: Predicted class {pred_class}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Image index 0: Predicted class 0
Image index 1: Predicted class 0
Image index 2: Predicted class 0
Image index 3: Predicted class 0
Image index 4: Predicted class 0
Image index 5: Predicted class 0
Image index 6: Predicted class 0
Image index 7: Predicted class 0
Image index 8: Predicted class 0
Image index 9: Predicted class 0
Image index 10: Predicted class 0
Image index 11: Predicted class 2
Image index 12: Predicted class 0
Image index 13: Predicted class 0
Image index 14: Predicted class 0
Image index 15: Predicted class 0
Image index 16: Predicted class 0
Image index 17: Predicted class 0
Image index 18: Predicted class 0
Image index 19: Predicted class 0
Image index 20: Predicted class 0
Image index 21: Predicted class 0
Image index 22: Predicted class 0
Image index 23: Predicted class 0
Image index 24: Predicted class 0
Image index 25: Predicted class 0
Image index 26: Predicted class 0
Image index 27: Predicted class 1
Image index 28: Pred

In [45]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the path to your training data
train_data_dir = 'data/train'

# Create an ImageDataGenerator for data augmentation and normalization
data_generator = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # Specify the validation split
)

# Load all data into a single generator with a validation split
all_data = data_generator.flow_from_directory(
    train_data_dir,
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical',
    shuffle=True,
    subset='training'  # Use the training subset
)

# Split the data into training and validation sets
train_data, val_data = train_test_split(all_data, test_size=0.2, random_state=42)

# Train the model using the training data and validate it using the validation data
classifier.fit(
    train_data[0],  # Training images
    train_data[1],  # Training labels
    steps_per_epoch=len(train_data[0]),
    epochs=25,
    validation_data=(val_data[0], val_data[1]),  # Validation data
    validation_steps=len(val_data[0])
)


Found 5716 images belonging to 3 classes.
Epoch 1/25


ValueError: Arguments `target` and `output` must have the same rank (ndim). Received: target.shape=(16, 64, 64, 3), output.shape=(16, 3)

In [42]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# Train the model using the training data
classifier.fit(
    train_data,
    steps_per_epoch=len(train_data),
    epochs=25,
    validation_data=val_data,
    validation_steps=len(val_data)
)

# Evaluate the model on the testing data to get testing accuracy
test_loss, test_accuracy = classifier.evaluate(validation_set)
print("Testing Accuracy:", test_accuracy)

# Generate predictions on the testing data
predictions = classifier.predict(validation_set)
predicted_classes = np.argmax(predictions, axis=1)

# Get true labels of testing data
true_classes = validation_set.classes

# Compute confusion matrix
conf_matrix = confusion_matrix(true_classes, predicted_classes)
print("Confusion Matrix:")
print(conf_matrix)

# Visualize confusion matrix
plt.figure(figsize=(8, 6))
plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.xticks(np.arange(len(classes)), classes, rotation=45)
plt.yticks(np.arange(len(classes)), classes)
plt.tight_layout()
plt.show()

# Identify misclassified images
misclassified_idx = np.where(predicted_classes != true_classes)[0]
misclassified_images = [validation_set.filenames[i] for i in misclassified_idx]
misclassified_labels = [classes[predicted_classes[i]] for i in misclassified_idx]

print("Misclassified Images:")
for image, label in zip(misclassified_images, misclassified_labels):
    print(f"Image: {image}, Predicted Label: {label}")


ValueError: Data is expected to be in format `x`, `(x,)`, `(x, y)`, or `(x, y, sample_weight)`, found: ((array([[[[0.57254905, 0.627451  , 0.67058825],
         [0.57254905, 0.627451  , 0.67058825],
         [0.57254905, 0.627451  , 0.67058825],
         ...,
         [0.82745105, 0.854902  , 0.82745105],
         [0.82745105, 0.854902  , 0.82745105],
         [0.82745105, 0.854902  , 0.82745105]],

        [[0.57254905, 0.627451  , 0.67058825],
         [0.57254905, 0.627451  , 0.67058825],
         [0.57254905, 0.627451  , 0.67058825],
         ...,
         [0.82745105, 0.854902  , 0.82745105],
         [0.82745105, 0.854902  , 0.82745105],
         [0.82745105, 0.854902  , 0.82745105]],

        [[0.57271135, 0.6276133 , 0.670426  ],
         [0.57271135, 0.6276133 , 0.670426  ],
         [0.57271135, 0.6276133 , 0.670426  ],
         ...,
         [0.82745105, 0.8547397 , 0.8277756 ],
         [0.82745105, 0.8547397 , 0.8277756 ],
         [0.82745105, 0.8547397 , 0.8277756 ]],

 